PROGRAMACIÓN DE LA LÓGICA PROGRAMABLE E IMPORTACIÓN DE LIBRERÍAS

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from time import sleep

#Esta librería permite la generación de números aleatorios
import random

DECLARACIÓN DE LOS ELEMENTOS DE LA PLACA

In [2]:
#Se accederá a LEDs, botones e interruptores aplicando lo mismo que en el proyecto anterior.
MAX_LEDS=4
leds=[base.leds[index] for index in range(MAX_LEDS)]

MAX_SW=2
switches=[base.switches[index] for index in range(MAX_SW)]

MAX_BTN=4
buttons=[base.buttons[index] for index in range(MAX_BTN)]

MAX_RGB=2
RGB=[base.rgbleds[index] for index in range(MAX_LEDS, MAX_LEDS+MAX_RGB)]

# PROGRAMA

SIMON

In [4]:
#Argumento para encender los RGBs en el color verde y rojo: RGB[X].write(verde/rojo)
verde, rojo = 2, 4

#Se comprueba en el archivo Record.txt el valor máximo
archivo=open('Record.txt', 'r')
record=archivo.readlines()
archivo.close()

while True:
    lista=[]
    contador=1
    Sigue, Bien = True, True

    #Se recorren uno a uno todos los LEDs y se apagan.
    for i in range(MAX_LEDS): leds[i].off()
    for i in range(MAX_RGB): RGB[i].write(0)

    sleep(.5)
    #Se realiza una secuencia de inicio para informar al usuario de que el juego va a comenzar
    for j in range(6):
        for i in range(MAX_LEDS): 
            leds[i].toggle()
            sleep(.125)
    for i in range(MAX_LEDS): leds[i].off()

    while Sigue:
        #Se genera un número aleatorio entre 0 y 3. Este número indica el LED que se añade a la secuencia
        lista.append(random.randint(0,3))
        #Se va recorriendo la secuencia encendiendo el LED correspondiente.
        for i in range(contador):
            leds[lista[i]].on()
            sleep(.5)
            leds[lista[i]].off()
            sleep(0.25)

        #Para cada uno de los elementos de la secuencia se comprueba si el usuario enciende el LED adecuado.
        for i in range(contador):
            #Se espera a que el usuario pulse un botón.
            while Sigue:
                for j in range(4):
                    if buttons[j].read(): 
                        leds[j].on()
                        Sigue=False
                        sleep(.25)
                        break
                        
            #Si el botón pulsado no coincide con el valor de la secuencia, se informa al usuario (a través de los LEDs) y se abandonan los bucles.
            if j!=lista[i]:
                leds[j].off()
                RGB[0].write(rojo)
                RGB[1].write(rojo)
                sleep(1)
                break

            Sigue=True
            leds[j].off()
            sleep(.5)

        #Si la secuencia introducida fue correcta, se avisa al usuario (con los LEDs) y se añade una posición a la secuencia
        if Sigue:
            RGB[0].write(verde)
            RGB[1].write(verde)
            sleep(1)
            RGB[0].write(0)
            RGB[1].write(0)
            contador+=1
            if (contador-1)%3==0 and contador!=1: print('Llevas ' + str(contador-1) + ' puntos.')
            
            
    print('Tu puntuación es de: ' + str(contador-1) + ' puntos.')
    #Se comprueba si se superó algún récord
    if (contador-1)>int(record[5]):
        i=5
        record[4:6]=[]
        print("¡Batiste un récord!")
        if (contador-1)>int(record[1]):
            i=1
        elif (contador-1)>int(record[3]):
            i=3
        print("Estás en " + str((i+1)//2) + "º posición.")
        archivo=open('Record.txt', 'w')
        record.insert(i-1, input('Cómo te llamas: ') + '\n')
        record.insert(i, str((contador-1)) + '\n')
        for i in range(6):
            archivo.write(record[i])
        archivo.close()
            
    #Cuando el juego finaliza, se espera una pulsación de un switch.
    while not switches[0].read() and switches[1].read():
        sleep(0.5)
    
    #El switch 0 apaga el juego. El 1 lo reinicia.
    if not switches[0].read(): break
        
for i in range(MAX_RGB): RGB[i].off()
for i in range(MAX_LEDS): leds[i].on()

Llevas 3 puntos.
Llevas 6 puntos.
Llevas 9 puntos.
Llevas 12 puntos.
Llevas 15 puntos.
Tu puntuación es de: 17 puntos.


COMPROBACIÓN DE LOS RÉCORDS

Si se accede al archivo Record.txt se podrá comprobar que la estructura de este es:
    Nombre1
    Puntuacion1
    Nombre2
    Puntuacion2
    Nombre3
    Puntuacion3

De esta forma, con la instrucción readlines(), almacenamos en record cada una de las líneas, permitiéndonos acceder a
ellas de forma independiente. A la hora de imprimirla en pantalla, se emplea rstrip() para eliminar el salto de línea.

In [3]:
#Para mostrar el TOP 3 se abre el archivo en modo lectura y se van recorriendo las líneas del .txt
archivo=open('Record.txt', 'r')
record=archivo.readlines()
archivo.close()
print('El récord lo tiene ' + record[0].rstrip('\n') + ' con ' + record[1].rstrip('\n') + ' acierto(s).')
print('La segunda posición es de ' + record[2].rstrip('\n') + ' con ' + record[3].rstrip('\n') + ' acierto(s).')
print('El bronce es para ' + record[4].rstrip('\n') + ' con ' + record[5].rstrip('\n') + ' acierto(s).')

El récord lo tiene AJDL con 24 acierto(s).
La segunda posición es de AJDL con 24 acierto(s).
El bronce es para AJDL con 17 acierto(s).
